In [114]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd

In [115]:
# 카뎃 목록 지정
cadets = "suhshin dha"

In [116]:
# notion api key 가져오기
load_dotenv()
my_bearer = os.getenv("BEARER")

In [117]:
libraryDbUrl = "https://api.notion.com/v1/databases/94e689f5b19947b1ab0f9b5f2d52962b/query"
contributorDbUrl = "https://api.notion.com/v1/databases/10909fc406ab42c29fa7f82d6a08d741/query"

In [118]:
headers = {
    "Authorization": "Bearer {}".format(my_bearer),
    "Accept": "application/json",
    "Notion-Version": "2022-02-22",
    "Content-Type": "application/json"
}

In [119]:
libraryBody = {
    "filter": {
        "and": [
            {
                "property": "create_date",
                "created_time": {
                    "on_or_after": "2022-01-01",
                }
            },
            {
                "property": "visible",
                "select": {
                    "does_not_equal": "no"
                }
            }
        ]
    },
    "page_size": 100
}

In [120]:
contributorBody = {
    "filter": {
        "property": "구분",
        "multi_select" : {
            "contains": "2기"
        }
    }
}

In [121]:
contributorResponse = requests.request("POST", contributorDbUrl, headers=headers, data=json.dumps(contributorBody))
contributorObject = json.loads(contributorResponse.text)

In [125]:
print(contributorObject["results"][0]["properties"])

{'\x08email': {'id': 'Glaa', 'type': 'rich_text', 'rich_text': [{'type': 'text', 'text': {'content': 'kikititi0510@gmail.com', 'link': {'url': 'kikititi0510@gmail.com'}}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'kikititi0510@gmail.com', 'href': 'kikititi0510@gmail.com'}]}, '\x08노션계정': {'id': 'NPPd', 'type': 'people', 'people': [{'object': 'user', 'id': '53ddec06-8d84-4d76-a487-f1300635abed'}]}, '구분': {'id': '%60%3DBq', 'type': 'multi_select', 'multi_select': [{'id': '106c3f1e-ed6a-4d93-9804-c09a04c7ea88', 'name': '2기', 'color': 'yellow'}]}, 'Library DB (속성)에 관계됨': {'id': 'olLW', 'type': 'relation', 'relation': []}, '이름': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'sohlee', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'sohlee', 'href': None

In [59]:
contributorList = []
for contributor in contributorObject["results"]:
    contributorList.append(contributor["properties"]["이름"]['title'][0]["plain_text"])

In [60]:
print(contributorList)
print(len(contributorList))

['sohlee', 'dohyukim', 'mingkim', 'sumkim', 'go', 'jchin', 'minjkim2', 'junhelee', 'suhyoon', 'seowokim', 'jmaing', 'siopark', 'taehyeok', 'jongpark', 'guntkim', 'eheo', 'woojlee', 'jungchoi', 'chaeyhan', 'hkong', 'mobae', 'ebang', 'hehwang', 'sooyokim', 'isong', 'mkwon', 'jayoon', 'jeongkpa', 'sangtale', 'koh', 'sujpark', 'mikim3', 'minkyeki', 'bolee', 'surlee', 'jijeong', 'jekim', 'solim', 'jiwhan', 'soylee', 'yunselee', 'hyeonjan', 'hyunhkim', 'mingylee', 'jbak', 'byeonkim', 'chanhpar', 'sunhwang', 'eukwon', 'jeykim', 'seohyuki', 'jaemjeon', 'junsoh', 'jchoi', 'sanghyel', 'chwang', 'sungjpar', 'seunpark', 'jibang', 'daejlee', 'yeblee', 'joonhan', 'dllee', 'hajeong', 'soopark', 'yomin', 'youngcho', 'kko', 'chanhale', 'ccho', 'yotak', 'minshin']
72


In [61]:
libraryResponse = requests.request("POST", libraryDbUrl, headers=headers, data=json.dumps(libraryBody))
libraryObject = json.loads(libraryResponse.text)
libraryObjectList = []
libraryObjectList.extend(libraryObject["results"])
print(len(libraryObjectList))

100


In [127]:
print(libraryObjectList[0]["properties"]["visible"]["select"])

None


In [63]:
print(libraryObject["next_cursor"])

c116ce03-2170-40d2-9706-dd9afa75e9f6


In [65]:
libraryBody["start_cursor"] = "c116ce03-2170-40d2-9706-dd9afa75e9f6"

In [66]:
print(libraryBody)

{'filter': {'and': [{'property': 'create_date', 'created_time': {'on_or_after': '2022-01-01'}}, {'property': 'visible', 'select': {'does_not_equal': 'no'}}]}, 'page_size': 100, 'start_cursor': 'c116ce03-2170-40d2-9706-dd9afa75e9f6'}


In [73]:

def filterByContributors(x, cadet):
    return cadet == x["properties"]["rich_text"]["formula"]["string"]

In [95]:
resultList = []
for contributor in contributorList:
        result = [x for x in libraryObjectList if filterByContributors(x, contributor)]
        resultList.append([contributor, len(result)])

In [109]:
df = pd.DataFrame(resultList)
df.columns = ['contributor', 'yes']
df["yes"].astype(int)

0     0
1     0
2     2
3     2
4     0
     ..
67    0
68    3
69    0
70    0
71    0
Name: yes, Length: 72, dtype: int64

In [110]:
mask = (df.yes >= 2)

In [111]:
df_done = df.loc[mask,:]
df_done.sort_values(by='yes',axis=0, ascending=False)

,contributor,yes
10,jmaing,10
61,joonhan,7
30,sujpark,6
28,sangtale,3
40,yunselee,3
68,chanhale,3
2,mingkim,2
3,sumkim,2
6,minjkim2,2
16,woojlee,2
